In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [2]:
all_matches = []

In [3]:
standings_url = "https://fbref.com/en/comps/9/2022-2023/2022-2023-Premier-League-Stats"

In [4]:
data = requests.get(standings_url)
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]

links = [l.get("href") for l in standings_table.find_all('a')]
links = [l for l in links if '/squads/' in l]
team_urls = [f"https://fbref.com{l}" for l in links]

In [8]:
from io import StringIO
import time
from functools import reduce

In [9]:

for team_url in team_urls:
    team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
    team_data = requests.get(team_url)
    try:
      matches = pd.read_html(StringIO(str(team_data.text)), match="Scores & Fixtures")[0]
      matches = matches[matches["Comp"] == "Premier League"]
    except ValueError:
        continue
    soup = BeautifulSoup(team_data.text)
    links = [l.get("href") for l in soup.find_all('a')]
    links = [l for l in links if l and 'all_comps/shooting/' in l]
    sh_link = links[0].split('/')[:-3] + ['c9', 'shooting', f'{team_name}-Match-Logs-Premier-League']
    shooting_data = requests.get(f"https://fbref.com{'/'.join(sh_link)}")
    pas_link = links[0].split('/')[:-3] + ['c9', 'passing', f'{team_name}-Match-Logs-Premier-League']
    passing_data = requests.get(f"https://fbref.com{'/'.join(pas_link)}")
    def_link = links[0].split('/')[:-3] + ['c9', 'defense', f'{team_name}-Match-Logs-Premier-League']
    defense_data = requests.get(f"https://fbref.com{'/'.join(def_link)}")
    poss_link = links[0].split('/')[:-3] + ['c9', 'possession', f'{team_name}-Match-Logs-Premier-League']
    possession_data = requests.get(f"https://fbref.com{'/'.join(poss_link)}")
    try:
        shooting = pd.read_html(StringIO(str(shooting_data.text)), match="Shooting")[0]
    except ValueError:
        continue
    try:
        passing = pd.read_html(StringIO(str(passing_data.text)), match="Passing")[0]
    except ValueError:
        continue
    try:
        defense = pd.read_html(StringIO(str(defense_data.text)), match="Defensive Actions")[0]
    except ValueError:
        continue
    try:
        possession = pd.read_html(StringIO(str(possession_data.text)), match="Possession")[0]
    except ValueError:
        continue
    shooting.columns = shooting.columns.droplevel()
    passing.columns = passing.columns.droplevel()
    defense.columns = defense.columns.droplevel()
    possession.columns = possession.columns.droplevel()
    try:
        shooting_data = shooting[["Date", "Sh", "SoT", "Dist", "SoT%", "G/Sh", "G/SoT", "npxG", "npxG/Sh", "G-xG", "np:G-xG"]]
        passing_data = passing[["Date", "Cmp", "Att", "Cmp%", "TotDist", "PrgDist", "Ast", "xAG", "xA", "KP"]]
        defense_data = defense[["Date", "Tkl", "TklW","Def 3rd","Mid 3rd", "Att 3rd", "Blocks", "Sh", "Pass", "Int", "Tkl+Int", "Clr", "Err"]]
        possession_data = possession[["Date", "Touches", "Def Pen", "Def 3rd", "Mid 3rd", "Att 3rd", "Att Pen", "Live"]]
        data_frames = [matches, shooting_data, passing_data, defense_data, possession_data]
        
        teams_data = reduce(lambda  left,right: pd.merge(left,right,on='Date'), data_frames)
        # matches1 = matches.merge(shooting, on="Date")
        # matches2 = matches1.merge(passing, on="Date")
        # matches3 = matches2.merge(defense, on="Date")
        # teams_data = matches3.merge(possession, on="Date")
    except ValueError:
        continue

    teams_data["Season"] = "2022-2023"
    teams_data["Team"] = team_name
    all_matches.append(teams_data)
    time.sleep(1)

In [12]:
match_df = pd.concat(all_matches)
match_df.columns = [c.lower() for c in match_df.columns]

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,err,touches,def pen,def 3rd_y,mid 3rd_y,att 3rd_y,att pen,live,season,team
0,2022-08-07,16:30,Premier League,Matchweek 1,Sun,Away,W,2,0,West Ham,...,0,936,55,227,484,234,35,935,2022-2023,Manchester City
1,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4,0,Bournemouth,...,0,814,20,83,409,325,32,814,2022-2023,Manchester City
2,2022-08-21,16:30,Premier League,Matchweek 3,Sun,Away,D,3,3,Newcastle Utd,...,0,756,66,196,364,202,36,756,2022-2023,Manchester City
3,2022-08-27,15:00,Premier League,Matchweek 4,Sat,Home,W,4,2,Crystal Palace,...,0,894,28,111,422,364,45,894,2022-2023,Manchester City
4,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,6,0,Nott'ham Forest,...,0,902,36,152,499,256,33,902,2022-2023,Manchester City
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33,2023-04-30,14:00,Premier League,Matchweek 34,Sun,Away,L,1,3,Newcastle Utd,...,1,485,107,236,190,68,13,485,2022-2023,Southampton
34,2023-05-08,20:00,Premier League,Matchweek 35,Mon,Away,L,3,4,Nott'ham Forest,...,0,688,48,159,367,166,37,687,2022-2023,Southampton
35,2023-05-13,15:00,Premier League,Matchweek 36,Sat,Home,L,0,2,Fulham,...,0,475,71,157,224,100,11,475,2022-2023,Southampton
36,2023-05-21,14:00,Premier League,Matchweek 37,Sun,Away,L,1,3,Brighton,...,1,499,110,268,160,76,5,499,2022-2023,Southampton


In [14]:
match_df.to_csv("matches22-23.csv")